In [1]:
%load_ext autoreload
%autoreload 2
#os.environ["OMP_NUM_THREADS"] = str(os.cpu_count() // 2)
#os.environ["GOMP_CPU_AFFINITY"] = "granularity=core,compact"
from Scripts.model import *
from Scripts.asymmetric_model import *
from Scripts.loss import *
from Scripts.results_manager import *
from Scripts.plots import *
from Scripts.dataset import *
from Scripts.trainer import *
from Scripts.inference import *
from Scripts.Onnx_Class import *
from Scripts.lr_finder import *
from Scripts.generate_configs import *
#from Scripts.asymmetric_model import *
from Scripts.generate_asymmetric_configs import *
from Scripts.excecute import *
#from Scripts.quantize import *
from Scripts.quantize_new import *
from Scripts.upload_summaries import *

c:\Users\alant\AppData\Local\Programs\Python\Python313\Lib\site-packages\openvino\runtime\__init__.py:10: DeprecationWarning: The `openvino.runtime` module is deprecated and will be removed in the 2026.0 release. Please replace `openvino.runtime` with `openvino`.
  warnings.warn(


In [ ]:
config_path = "Asymmetric_Configs"

In [ ]:
training_selected_class(config_path, 'grid')
# metrics_to_db()

In [ ]:
training_all_classes(config_path)
metrics_to_db()

In [ ]:
training_run_folder = "Training_Runs"
inference_output_dir = "Inference_Runs"

inference_model(training_run_folder, inference_output_dir)

In [ ]:
config_paths = glob.glob(os.path.join(
    "Training_Runs", "**", "*.yaml"), recursive=True)
summary_metrics_paths = glob.glob(os.path.join(
    "Training_Runs", "**", "summary_metrics.json"), recursive=True)
best_student_weight_paths = glob.glob(os.path.join(
    "Training_Runs", "**", "*best_student.pth"), recursive=True)
for configs, summary_metrics, best_student_weights in zip(config_paths, summary_metrics_paths, best_student_weight_paths):
    config = load_config(configs)
    summary_metric = load_json(summary_metrics)
    quantize_model(
        best_student_weights,
        config,
        summary_metric
    )

In [2]:
inference_output_dir = 'quantized_inference_results'
cpu_device = torch.device("cpu")

# Finde alle quantisierten Modelle im Ordner
quantized_weight_paths = glob.glob(os.path.join(
    "quantized_models", "**", "quantized_model.pth"), recursive=True)

print(
    f"Führe Inferenz für {len(quantized_weight_paths)} quantisierte Modelle aus...")

for weight_path in quantized_weight_paths:
    dirpath = os.path.dirname(weight_path)
    print(f"\n--- Verarbeite Modell in: {dirpath} ---")

    # Finde die zugehörigen Konfigurationsdateien
    yaml_filename = None
    for file in os.listdir(dirpath):
        if file.endswith('.yaml'):
            yaml_filename = file
            break

    if yaml_filename is None:
        print(
            f"Warnung: Keine YAML-Konfigurationsdatei in {dirpath} gefunden. Überspringe.")
        continue

    json_path = os.path.join(dirpath, "summary_metric.json")
    yaml_path = os.path.join(dirpath, yaml_filename)

    if not os.path.exists(json_path):
        print(
            f"Warnung: Keine summary_metric.json in {dirpath} gefunden. Überspringe.")
        continue

    # Lade Konfigurationen
    config_data = load_config(yaml_path)
    summary_data = load_json(json_path)
    training_id = summary_data.get(
        'training_id', 'quantized_run')  # Nutze Fallback

    # --- WICHTIGER TEIL: Lade das quantisierte Modell korrekt ---

    # 1. Erstelle das leere STFPM-Modellgerüst
    inference_model = STFPM(
        architecture=config_data['model']['architecture'],
        layers=config_data['model']['layers']
    )
    inference_model.eval()
    student_model_to_quantize = inference_model.student_model
    student_model_to_quantize.eval()

    # 2. Bereite das student_model mit der exakt gleichen FX-Konfiguration vor
    qconfig_mapping = get_default_qconfig_mapping("fbgemm")
    example_inputs = (torch.randn(1, 3, 256, 256),)
    prepared_model = prepare_fx(
        student_model_to_quantize, qconfig_mapping, example_inputs)
    quantized_student_model = convert_fx(prepared_model)

    # 3. Lade jetzt die gespeicherten, quantisierten Gewichte
    quantized_student_model.load_state_dict(
        torch.load(weight_path, map_location=cpu_device))
    quantized_student_model.eval()
    print("Quantisierte Gewichte erfolgreich geladen.")

    # 4. Setze das quantisierte student_model in das STFPM-Gesamtmodell ein
    inference_model.student_model = quantized_student_model
    inference_model.to(cpu_device)

    # --- Ab hier läuft die Inferenz wie gewohnt ---

    # Lade die Testdaten
    try:
        test_set = MVTecDataset(
            img_size=config_data['dataset']['img_size'],
            base_path=config_data['dataset']['base_path'],
            cls=config_data['dataset']['class'],
            mode='test',
            download_if_missing=False
        )
        test_loader = DataLoader(
            test_set,
            batch_size=config_data['dataloader']['batch_size'],
            shuffle=False
        )
    except Exception as e:
        print(f"Fehler beim Laden des Test-Datensatzes für {yaml_path}: {e}")
        continue

    # Führe die Inferenz aus
    infer = Inference(
        model=inference_model,
        test_loader=test_loader,
        config=config_data,
        output_dir=inference_output_dir,
        path_to_student_weight=None,
        trainings_id=training_id,
        inferenz=True
    )

    print(f"Starte Inferenz für Konfiguration: {yaml_path}...")
    auroc_score, total_inference_time = infer.evaluate_loaded_model()
    infer.create_inference_summary(
        summary_data, auroc_score, total_inference_time)
    print(
        f"Inferenz abgeschlossen. AUROC: {auroc_score:.4f}, Zeit: {total_inference_time:.4f}s.")

    infer.generate_heatmaps_from_saved_maps()

print("\n--- Alle Inferenzläufe abgeschlossen. ---")

Führe Inferenz für 5 quantisierte Modelle aus...

--- Verarbeite Modell in: quantized_models\mobilenetv3_large_100\f07b3762-1522-4748-bb8c-60cfbcfa621b ---


Unexpected keys (classifier.bias, classifier.weight, conv_head.bias, conv_head.weight) found while loading pretrained weights. This may be expected if model is being adapted.
C:\Users\alant\AppData\Local\Temp\ipykernel_27532\124296653.py:55: DeprecationWarning: torch.ao.quantization is deprecated and will be removed in 2.10. 
For migrations of users: 
1. Eager mode quantization (torch.ao.quantization.quantize, torch.ao.quantization.quantize_dynamic), please migrate to use torchao eager mode quantize_ API instead 
2. FX graph mode quantization (torch.ao.quantization.quantize_fx.prepare_fx,torch.ao.quantization.quantize_fx.convert_fx, please migrate to use torchao pt2e quantization API instead (prepare_pt2e, convert_pt2e) 
3. pt2e quantization has been migrated to torchao (https://github.com/pytorch/ao/tree/main/torchao/quantization/pt2e) 
see https://github.com/pytorch/ao/issues/2259 for more details
  prepared_model = prepare_fx(
c:\Users\alant\AppData\Local\Programs\Python\Python313\L

Quantisierte Gewichte erfolgreich geladen.
Starte Inferenz für Konfiguration: quantized_models\mobilenetv3_large_100\f07b3762-1522-4748-bb8c-60cfbcfa621b\STFPM_Config_mobilenetv3_large_100.yaml...


Inference summary saved to quantized_inference_results\MVTecAD_bottle\mobilenetv3_large_100\f07b3762-1522-4748-bb8c-60cfbcfa621b\inference_summary.json
Inferenz abgeschlossen. AUROC: 0.9865, Zeit: 1.2477s.

Starte Heatmap-Generierung aus gespeicherten Karten für: quantized_inference_results\MVTecAD_bottle\mobilenetv3_large_100\f07b3762-1522-4748-bb8c-60cfbcfa621b


Erstelle Heatmaps: 100%|██████████| 83/83 [00:19<00:00,  4.23it/s]


...

C:\Users\alant\AppData\Local\Temp\ipykernel_27532\124296653.py:55: DeprecationWarning: torch.ao.quantization is deprecated and will be removed in 2.10. 
For migrations of users: 
1. Eager mode quantization (torch.ao.quantization.quantize, torch.ao.quantization.quantize_dynamic), please migrate to use torchao eager mode quantize_ API instead 
2. FX graph mode quantization (torch.ao.quantization.quantize_fx.prepare_fx,torch.ao.quantization.quantize_fx.convert_fx, please migrate to use torchao pt2e quantization API instead (prepare_pt2e, convert_pt2e) 
3. pt2e quantization has been migrated to torchao (https://github.com/pytorch/ao/tree/main/torchao/quantization/pt2e) 
see https://github.com/pytorch/ao/issues/2259 for more details
  prepared_model = prepare_fx(
c:\Users\alant\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\ao\quantization\observer.py:246: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range wi

Quantisierte Gewichte erfolgreich geladen.
Starte Inferenz für Konfiguration: quantized_models\resnet18\22a1ec46-6ca8-42ec-8889-6a2956edf095\STFPM_Config_resnet18.yaml...


Inference summary saved to quantized_inference_results\MVTecAD_bottle\resnet18\22a1ec46-6ca8-42ec-8889-6a2956edf095\inference_summary.json
Inferenz abgeschlossen. AUROC: 1.0000, Zeit: 1.4990s.

Starte Heatmap-Generierung aus gespeicherten Karten für: quantized_inference_results\MVTecAD_bottle\resnet18\22a1ec46-6ca8-42ec-8889-6a2956edf095


Erstelle Heatmaps: 100%|██████████| 83/83 [00:19<00:00,  4.20it/s]

Heatmap-Generierung abgeschlossen. Ergebnisse in 'quantized_inference_results\MVTecAD_bottle\resnet18\22a1ec46-6ca8-42ec-8889-6a2956edf095\plots' gespeichert.

--- Alle Inferenzläufe abgeschlossen. ---


In [ ]:
# Lade Konfiguration und Metriken
config = load_config(
    r'Training_Runs\MVTecAD_grid\resnet18\2830be19-bea8-41f8-a1e8-1a6ef49b8cbc\STFPM_Config_resnet18.yaml')
cpu_device = torch.device("cpu")
summary_metrics = load_json(
    r'Training_Runs\MVTecAD_grid\resnet18\2830be19-bea8-41f8-a1e8-1a6ef49b8cbc\summary_metrics.json')

# 1. Quantisiere NUR das student_model mit deiner neuen FX-Funktion
quantized_student_model = quantize_model(
    r'Training_Runs\MVTecAD_grid\resnet18\2830be19-bea8-41f8-a1e8-1a6ef49b8cbc\weights\resnet18_grid_2830be19-bea8-41f8-a1e8-1a6ef49b8cbc_best_student.pth',
    config
)

# 2. Erstelle ein neues, vollständiges STFPM-Modell für die Inferenz
inference_model = STFPM(
    architecture=config['model']['architecture'],
    layers=config['model']['layers']
)

# 3. Ersetze das untrainierte student_model durch deine quantisierte Version
inference_model.student_model = quantized_student_model
inference_model.to(cpu_device).eval()


# Lade die Testdaten
test_set = MVTecDataset(
    img_size=config['dataset']['img_size'],
    base_path=config['dataset']['base_path'],
    cls=config['dataset']['class'],
    mode='test',
    download_if_missing=False
)
test_loader = DataLoader(
    test_set,
    batch_size=config['dataloader']['batch_size'],
    shuffle=False
)

# 4. Übergebe das vollständige, zusammengesetzte Modell an die Inference-Klasse
infer = Inference(
    model=inference_model,  # <--- Hier wird das korrekte Modell übergeben
    test_loader=test_loader,
    config=config,
    path_to_student_weight=None,  # Gewichte sind schon geladen
    trainings_id=summary_metrics.get('training_id'),
    inferenz=True
)

# 5. Führe die Inferenz aus
auroc, inf_time = infer.evaluate_per_epoch()
print(f"AUROC: {auroc}, Inference Time: {inf_time}")

In [ ]:
import onnxruntime as ort
import numpy as np
from PIL import Image

# 1. ONNX-Modell laden
onnx_model_path = r"onnx_models\STFPM_resnet18.onnx"
sess = ort.InferenceSession(onnx_model_path)

image_path = glob.glob(os.path.join(
    r'Images\bottle\test\broken_large', '*.png'))
for path in image_path:

    # 2. Ein Bild für die Inferenz vorbereiten
    # image_path = r"Images\grid\test\bent\003.png"
    image = Image.open(path).convert("RGB")

    # Resize the image to the expected input size
    img_size = 256
    image = image.resize((img_size, img_size))

    input_data = np.array(image, dtype=np.uint8)
    input_data = np.expand_dims(input_data, axis=0)  # Add batch dimension

    # 3. Inferenz durchführen
    input_name = sess.get_inputs()[0].name
    outputs = sess.run(None, {input_name: input_data})

    # 4. Ergebnisse auswerten
    anomaly_map = outputs[0]
    anomaly_score = outputs[1]

    print(f"Anomalie-Score für das Bild: {anomaly_score[0]}")

    fig, axes = plt.subplots(1, 2, figsize=(12, 6))

    # 2. Originalbild anzeigen
    axes[0].imshow(image)
    axes[0].set_title("Originalbild")
    axes[0].axis('off')

    # 3. Heatmap über das Originalbild legen
    axes[1].imshow(image)
    heatmap = axes[1].imshow(np.squeeze(anomaly_map), cmap='jet', alpha=0.5)
    axes[1].set_title("Anomalie-Heatmap")
    axes[1].axis('off')

    # 4. Farbbalken für die Heatmap hinzufügen
    fig.colorbar(heatmap, ax=axes[1], fraction=0.046, pad=0.04)

    plt.tight_layout(rect=[0, 0, 1, 0.95])
    # 5. Plot anzeigen oder speichern
    plt.suptitle("Vergleich: Originalbild vs. Anomalie-Heatmap")
    plt.show()